# Naive Bayes dengan Data Kontinu

Pada percobaan ketiga ini, kita akan menggunakan data riil untuk melakukan klasifikasi dengan Naive Bayes. Data yang digunakan adalah **Sports Probabilitas Cuaca Tennis**. Data tersebut menggambarkan usia seseorang dengan pendapatkan yang akan menentukan apakah orang tersebut akan membeli sebuah barang atau tidak. Gaji merupakan data kontinu.

Isikan identitas berikut <br>
**Nama  : Mochammad Hairullah** <br>
**NIM   : 2041720074** <br>
**Kelas : TI-3H**

## Tahap Persiapan
Pada tahap ini kita akan melakukan beberapa hal,

1. Load data ke dalam data frame
2. Memisahkan fitur dan label
3. Split data untuk training dan testing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd

# Load data CSV
df = pd.read_csv('./dataset/tennis-predict.csv')

# Cek data
display(df.head())

,outlook,temp,humidity,windy,play
0,sunny,hot,high,False,no
1,sunny,hot,high,True,no
2,overcast,hot,high,False,yes
3,rainy,mild,high,False,yes
4,rainy,cool,normal,False,yes


In [ ]:
# Encoding
# Fungsi encoding yang akan digunakan adalah LabelEncoder
# Hal ini karena kita hanya mengganti nilai variabel dari nama berupa string menjadi angka. Sama halnya dengan label

from sklearn.preprocessing import LabelEncoder

# Inisiasi label encoder
encode = LabelEncoder()

# Terpakan label encoder
df['outlook'] = encode.fit_transform(df['outlook'])
df['temp'] = encode.fit_transform(df['temp'])
df['humidity'] = encode.fit_transform(df['humidity'])
df['windy'] = encode.fit_transform(df['windy'])

# Cek hasil
df.head()

,outlook,temp,humidity,windy,play
0,2,1,0,0,no
1,2,1,0,1,no
2,0,1,0,0,yes
3,1,2,0,0,yes
4,1,0,1,0,yes


In [ ]:
# Memisahkan fitur dengan label
X = df.iloc[:,:-1]
y = df.iloc[:,-1]

print(X)

    outlook  temp  humidity  windy
0         2     1         0      0
1         2     1         0      1
2         0     1         0      0
3         1     2         0      0
4         1     0         1      0
5         1     0         1      1
6         0     0         1      1
7         2     2         0      0
8         2     0         1      0
9         1     2         1      0
10        2     2         1      1
11        0     2         0      1
12        0     1         1      0
13        1     2         0      1


In [ ]:
# Split data training dan testing

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=30)

## Training dan Evaluasi Model

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

# Inisiasi obyek MultinomialNB
gnb = GaussianNB()

# Fit model
# Label y harus dalam bentu 1D atau (n_samples,)
gnb.fit(X_train, y_train)

# Prediksi dengan data training
y_train_pred = gnb.predict(X_train)

# Evaluasi akurasi training
acc_train = accuracy_score(y_train, y_train_pred)

# Prediksi test data
y_test_pred = gnb.predict(X_test)

# Evaluasi model dengan metric akurasi
acc_test = accuracy_score(y_test, y_test_pred)

# Print hasil evaluasi
print(f'Hasil akurasi data train: {acc_train}')
print(f'Hasil akurasi data test: {acc_test}')

Hasil akurasi data train: 0.8888888888888888
Hasil akurasi data test: 1.0


## Apakah kita telah melakukan hal yang benar?

Perhatikan kembali fitur dari data. Terdapat **outlook** dan **temp**. Selain keduanya merupakan data kontinu, _outlook_ dan _temp_ memiliki skala yang berbeda. Hal ini "mungkin" dapat menyebabkan kurang akuratnya model dalam memprediksi sebuah kelas. Apakah ini benar?

## Pembuktian

Kita akan melakukan percobaan lanjutan dengan menggunakan standarisasi untuk menjawab pertanyaan, apakah kita perlu melakukan hal tersebut pada model Naive Bayes khususnya tipe Gaussian.

In [ ]:
from sklearn.preprocessing import StandardScaler

# Inisasi obyek StandardScaler
scaler = StandardScaler()

# Standarisasi pada fitur di X_train dan X_test
X_train_std = scaler.fit_transform(X_train)
X_test_std = scaler.transform(X_test)

***Pertanyaan***

Mengapa pada X_test kita tidak perlu melakukan proses fitting?

***Jawaban***

Proses fitting (fit) akan menyimpan perhitungan berdasarkan data yang dilatih. Pada kasus standarisasi ini, nilai yang disimpan adalah *mean* dan *standar deviasi* dari data **X_train**. Jika kita melakukan proses fitting kembali pada **X_test** maka nilai mean dan standar deviasi akan berdasarkan data X_test. Kita tidak ingin itu terjadi, karena model diharapkan mampu melakukan klasifikasi dengan baik pada data yang tidak diketahui (data test). Oleh karena itu, pada X_test hanya dilakukan proses **transform** agar pada saat pembuatan model, model akan menggunakan nilai mean dan standar deviasi yang sama dengan data training.

In [ ]:
# Buat obyek GaussianNB lain
gnb_std = GaussianNB()

# Fit dengan data yang telah di standarisasi
gnb_std.fit(X_train_std, y_train)

# Prediksi dengan data training
y_train_std_pred = gnb_std.predict(X_train_std)

# Evaluasi akurasi training data
acc_train_std = accuracy_score(y_train, y_train_std_pred)

# Prediksi test data yang telah di standarisasi
y_test_std_pred = gnb_std.predict(X_test_std)

# Evaluasi akurasi testing data
acc_test_std = accuracy_score(y_test, y_test_std_pred)

# Print hasil evaluasi
print(f'Hasil akurasi data training terstandarisasi: {acc_train_std}')
print(f'Hasil akurasi data testing terstandarisasi: {acc_test_std}')

Hasil akurasi data training terstandarisasi: 0.8888888888888888
Hasil akurasi data testing terstandarisasi: 1.0


## Kesimpulan

Jika diperhatikan, tidak terjadi perubahan yang signifikan antara model dari nilai asli dengan model dengan nilai yang telah di standarisasi, terlebih pada hasil dengan menggunakan data test. Hal ini dikarenakan, Naive Bayes bukan jenis algoritma klasifikasi yang mengandalkan jarak, namun probabilitas. Mean dan standar deviasi mungkin berubah, namun probabilitas akan menghasilkan nilai yang sama